# Long short-term memory (LSTM)
## Author: Taichi Nakamura (Keio university, Japan)
Taichi Nakamura provides no guarantees for this code.  Use as-is and for academic research use only; no commercial use allowed without permission. For citations, please use the reference below:  
Ref: Taichi Nakamura, Kai Fukami, and Koji Fukagata, "Convolutional neural network and long short-term memory based reduced order surrogate for minimal turbulent channel flow," Physics of Fluids, 2021.  
The code is written for educational clarity and not for speed.  
-- version 1: Jan 27, 2021

For making LSTM, the user has to install 'keras', 'numpy', 'pandas' and 'sklearn'.

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Add
from keras.layers.recurrent import LSTM
from keras.models import Model
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model

Using TensorFlow backend.


## Load data

In [ ]:
gridSetting = (34,66,34)

datasetSerial = np.arange(100,1000100,100)
datasetPrefix = './data_box/UVWP'

u_all = np.zeros((len(datasetSerial),32,32,32))
v_all = np.zeros((len(datasetSerial),32,32,32))
w_all = np.zeros((len(datasetSerial),32,32,32))

for i in tqdm(range(len(datasetSerial))):
    name = datasetPrefix + '{0:07d}'.format(datasetSerial[i])
    df = pd.read_csv(name, header=None, delim_whitespace=True)
    dataset = df.values
    BOX = dataset[:,:]

    u_org = BOX[:,0].reshape(gridSetting,order='F')
    v_org = BOX[:,1].reshape(gridSetting,order='F')
    w_org = BOX[:,2].reshape(gridSetting,order='F')

    u_all[i,:,:,:] = u_org[1:33,1:33,1:33]
    v_all[i,:,:,:] = v_org[1:33,1:33,1:33]
    w_all[i,:,:,:] = w_org[1:33,1:33,1:33]
    
u_mean = u_all.mean(axis=3).mean(axis=0).mean(axis=0)
v_mean = v_all.mean(axis=3).mean(axis=0).mean(axis=0)
w_mean = w_all.mean(axis=3).mean(axis=0).mean(axis=0)

u_flc = np.zeros(u_all.shape)
v_flc = np.zeros(v_all.shape)
w_flc = np.zeros(w_all.shape)

for j in tqdm(range(32)):
    u_flc[:,:,j,:] = u_all[:,:,j,:] - u_mean[j]
    v_flc[:,:,j,:] = v_all[:,:,j,:] - v_mean[j]
    w_flc[:,:,j,:] = w_all[:,:,j,:] - w_mean[j]

data_raw = np.zeros((len(datasetSerial),32,32,32,3))
data_raw[:,:,:,:,0] = u_flc
data_raw[:,:,:,:,1] = v_flc
data_raw[:,:,:,:,2] = w_flc

## Obtain latent vectors ${\eta}$

In [ ]:
model = load_model('./Model.hdf5') # Load CNN-AE
layer_name = 'reshape_1'
model_latent = Model(inputs=model.input, outputs=model.get_layer(layer_name).output) # Define CNN encoder part
latent_vec = model_latent.predict(data_raw) # (# of dataset, latent vector size)

## Make training dataset for LSTM  
input data: {$\eta^{(n-5)\Delta t},\eta^{(n-4)\Delta t},\eta^{(n-3)\Delta t},\eta^{(n-2)\Delta t},\eta^{(n-1)\Delta t}$}  
shape: (# of dataset, 5, latent vector size)  
label: {$\eta^{(n-4)\Delta t},\eta^{(n-3)\Delta t},\eta^{(n-2)\Delta t},\eta^{(n-1)\Delta t},\eta^{n\Delta t}$}  
shape: (# of dataset, 5, latent vector size)

In [ ]:
def make_dataset(data_in):

    data, target = [], [] # Input data, label
    maxlen = 5 # The number of timestep required for prediction

    for i in range(len(data_in[:,0])-maxlen):
        data.append(data_in[i:i + maxlen,:])
        target.append(data_in[(i+1):(i+maxlen+1),:])

    re_data = np.array(data).reshape(len(data), maxlen, 4*4*4*3,order='F')
    re_target = np.array(target).reshape(len(data), maxlen, 4*4*4*3,order='F')

    return re_data, re_target

data_input, data_label = make_dataset(latent_vec) # Input data, label

## Constrct LSTM model

In [ ]:
input_img = Input(shape=(None,192))

x1 = LSTM(192,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x1 = LSTM(192,return_sequences=True,activation='linear')(x1)

x2 = LSTM(288,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x2 = LSTM(192,return_sequences=True,activation='linear')(x2)

x3 = LSTM(384,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x3 = LSTM(192,return_sequences=True,activation='linear')(x3)

x4 = LSTM(480,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x4 = LSTM(192,return_sequences=True,activation='linear')(x4)

x5 = LSTM(576,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x5 = LSTM(192,return_sequences=True,activation='linear')(x5)

x6 = LSTM(672,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x6 = LSTM(192,return_sequences=True,activation='linear')(x6)

x7 = LSTM(768,return_sequences=True,activation='tanh',batch_input_shape=(None, None, 192))(input_img)
x7 = LSTM(192,return_sequences=True,activation='linear')(x7)

x = Add()([x1,x2,x3,x4,x5,x6,x7])
x_final = (Activation('tanh'))(x)

model_LSTM = Model(input_img, x_final)
model_LSTM.compile(optimizer='adam', loss='mse')

## Learn parameters

In [ ]:
model_cb=ModelCheckpoint('./Model_LSTM.hdf5', monitor='val_loss',save_best_only=True,verbose=1)
early_cb=EarlyStopping(monitor='val_loss', patience=50,verbose=1)
X_train, X_test, y_train, y_test = train_test_split(data_input, data_label, test_size=0.3, random_state=True)
cb = [model_cb, early_cb]
history = dscms.fit(X_train, y_train,
          batch_size=16,
          epochs=1000,
          verbose = 1,
          validation_data=[X_test,y_test],
          callbacks=cb,
          shuffle=True
          )
df_results = pd.DataFrame(history.history)
df_results['epoch'] = history.epoch
df_results.to_csv(path_or_buf='./Model_LSTM.csv',index=False)